In [61]:
import pandas as pd
from datetime import date
import json

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [26]:
death_words = ['DEATH','DEAD','DIED']

def clean_outcome(row):
    if row['outcome'] in death_words:
        return 'DEATH'
    else:
        return 'POSITIVE'

In [29]:
cnty_join = pd.read_csv('../data/tl_2019_us_county.csv',dtype={'STATEFP':str,'COUNTYFP':str,'GEOID':str})
cnty_join = cnty_join[['STATEFP','COUNTYFP','GEOID','NAMELSAD']]

state = 'California'
state_fips = '06'
today = str(date.today())
print("Today's date:", today)

case_data = pd.read_csv('../data/case-data/outside_Hubei.data.19032020T011105.csv')
us_cases = case_data.loc[case_data['country'] == 'United States']
state_cases = us_cases.loc[us_cases['province'] == state]
state_cases['date_confirm_clean'] = pd.to_datetime(state_cases['date_confirmation'], format='%d.%m.%Y', errors='ignore')

state_cases['outcome'] = state_cases['outcome'].str.upper()
state_cases['outcome_clean'] = state_cases.apply(lambda row: clean_outcome(row), axis=1)

#join county fips
cnty_join = cnty_join.loc[cnty_join['STATEFP'] == state_fips]
cnty_join = cnty_join[['NAMELSAD','COUNTYFP']]
cnty_join_dict = dict(zip(cnty_join.NAMELSAD, cnty_join.COUNTYFP))     
state_cases['fips'] = state_cases['city'].map(cnty_join_dict)

#pivot on county, agg deaths positive cases
by_county = pd.pivot_table(state_cases, values='ID', index=['province','fips','city','date_confirm_clean'],
                           columns=['outcome_clean'], aggfunc='count')

print('Total cases reported for',state,':',len(state_cases))
print('Total deaths reported for',state,':',len(state_cases.loc[state_cases['outcome_clean'] == 'DEATH']))
print('Most recent case confirmation date:',state_cases.date_confirm_clean.max())
display(by_county)
by_county.to_csv('../data/case-data/'+today+'-'+state+'-export.csv')

Today's date: 2020-03-20
Total cases reported for California : 471
Total deaths reported for California : 0
Most recent case confirmation date: 2020-03-16 00:00:00


/Users/akanik/.virtualenvs/covid19/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,22,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/akanik/.virtualenvs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/akanik/.virtualenvs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

outcome_clean                                              POSITIVE
province   fips city                   date_confirm_clean          
California 001  Alameda County         2020-03-01                 1
                                       2020-03-06                 1
                                       2020-03-10                 1
                                       2020-03-12                 2
                                       2020-03-15                11
                                       2020-03-16                 3
           009  Calaveras County       2020-03-10                 2
           013  Contra Costa County    2020-03-03                 1
                                       2020-03-06                 6
                                       2020-03-08                 5
                                       2020-03-12                 4
                                       2020-03-14                13
                                       2020-03-16                 5
           019  Fresno County          2020-03-10                 2
           023  Humboldt County        2020-02-20                 1
           025  Imperial County        2020-03-12                 2
           037  Los Angeles County     2020-03-06                 2
                                       2020-03-07                 1
                                       2020-03-11                 8
                                       2020-03-12                 7
                                       2020-03-13                 8
                                       2020-03-14                13
                                       2020-03-15                16
           039  Madera County          2020-03-07                 1
           041  Marin County           2020-03-09                 1
                                       2020-03-13                 2
                                       2020-03-14                 2
                                       2020-03-15                 4
           057  Nevada County          2020-03-16                 1
           059  Orange County          2020-01-26                 1
                                       2020-03-03                 2
                                       2020-03-08                 1
                                       2020-03-13                 2
                                       2020-03-14                 3
                                       2020-03-15                 4
           061  Placer County          2020-03-02                 1
                                       2020-03-03                 1
                                       2020-03-06                 3
                                       2020-03-08                 2
           065  Riverside County       2020-03-08                 1
                                       2020-03-09                 3
                                       2020-03-11                 1
                                       2020-03-13                 3
           067  Sacramento County      2020-02-21                 1
                                       2020-03-09                 4
                                       2020-03-10                 5
                                       2020-03-12                 6
                                       2020-03-14                12
           069  San Benito County      2020-02-02                 2
                                       2020-03-13                 1
                                       2020-03-14                 1
           071  San Bernardino County  2020-03-15                 1
           073  San Diego County       2020-02-11                 1
                                       2020-02-12                 1
                                       2020-03-08                 1
                                       2020-03-09                 1
                                       2020-03-13       